In [9]:
import numpy as np
import pandas as pd

In [10]:
np.array([1, 11, 6, 53, 66, 73, 74, 21, 96, 86,])-1

array([ 0, 10,  5, 52, 65, 72, 73, 20, 95, 85])

In [11]:
data = pd.read_csv("/content/drive/MyDrive/mydata/project/en_wiki_100.csv")
data.head()
# col_count = 10
epochs = 200
cols = list(data.columns)
sites = cols[1:]
data = data[sites]
data.head()

,"""Weird_Al""_Yankovic_en.wikipedia.org_desktop_all-agents",100_metres_en.wikipedia.org_desktop_all-agents,10_Gigabit_Ethernet_en.wikipedia.org_desktop_all-agents,1551_en.wikipedia.org_desktop_all-agents,1896_Summer_Olympics_en.wikipedia.org_desktop_all-agents,1918_flu_pandemic_en.wikipedia.org_desktop_all-agents,1936_Summer_Olympics_en.wikipedia.org_desktop_all-agents,1976_Summer_Olympics_en.wikipedia.org_desktop_all-agents,1980_Summer_Olympics_en.wikipedia.org_desktop_all-agents,1984_Summer_Olympics_en.wikipedia.org_desktop_all-agents,...,Alaska_en.wikipedia.org_desktop_all-agents,"Albert,_Prince_Consort_en.wikipedia.org_desktop_all-agents",Albert_Einstein_en.wikipedia.org_desktop_all-agents,Albus_Dumbledore_en.wikipedia.org_desktop_all-agents,Alex_(parrot)_en.wikipedia.org_desktop_all-agents,Alex_Jones_(radio_host)_en.wikipedia.org_desktop_all-agents,Alexander_I_of_Russia_en.wikipedia.org_desktop_all-agents,Alexandre_Pato_en.wikipedia.org_desktop_all-agents,Alexis_Arquette_en.wikipedia.org_desktop_all-agents,Alexis_Ohanian_en.wikipedia.org_desktop_all-agents
0,2403,982,1078,11,366,868,714,511,414,594,...,4590,933,8398,871,299,1383,596,1111,616,253
1,20136,881,1116,9,392,877,616,648,430,518,...,3545,902,8600,830,288,966,532,836,468,288
2,1850,794,869,19,322,688,506,431,368,441,...,3294,788,7474,896,282,865,407,846,416,10696
3,1432,979,327,26,313,623,504,387,365,425,...,2885,892,5900,1185,281,812,409,648,425,5672
4,1351,1191,312,16,340,604,579,386,411,409,...,3180,940,6835,878,297,863,541,719,585,2435


In [12]:
def split_seq_c(D, inp_size,test_size=100):
  start_indices = np.array(range(0,len(D)-test_size-inp_size))
  return [
    np.array(list(map(lambda i:D[i:i+inp_size], start_indices))),
    np.array(list(map(lambda i:D[i+inp_size], start_indices))),
    np.array(D[-test_size:]),
  ]

sequences = []
n_steps = 10
for site in sites:
    X, y, test_data = split_seq_c(data[site], n_steps)
    sequences.append({
        'X': X,
        'y': y,
        'test_data':test_data,
    })
len(sequences)

100

In [13]:
# for seq in sequences:
#     print('\n'.join([f'{x}: {y_}' for x,y_ in zip(seq['X'],seq['y'])]))

# seq = sequences[0]
# print('\n'.join([f'{x}: {y_}' for x,y_ in zip(seq['X'],seq['y'])]))

In [14]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional

Vanilla LSTM

In [15]:
vanillas = []

for seq in sequences:
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    vanillas.append(model)


In [16]:
for i in range(len(vanillas)):
    X = seq['X'].reshape(list(seq['X'].shape)+[1])
    vanillas[i].fit(X, seq['y'], epochs=epochs, verbose=0)
    print(f'{i+1} done')

1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done


In [17]:
def predict(model, x):
    x_inp = np.array(x)
    x_inp = x_inp.reshape(1,n_steps, 1)
    return model.predict(x_inp, verbose=0).reshape([])

In [18]:
def internal_error(model, X, y):
    y_pred = np.array([predict(model, x) for x in X])
    y_true = np.array(y)
    return np.mean(np.abs(y_true-y_pred))

# errors = []
# for i in range(len(sites)):
#     errors.append(internal_error(vanillas[i], sequences[i]['X'], sequences[i]['y']))
#     print(f"Error for site {i+1} : {errors[-1]}")

# print()
# print(np.sum(errors)/len(errors))

In [19]:
def external_error(model, last_x, test_data, verbose=0):
    x = last_x

    error_sum = 0
    percent_error_sum = 0
    for i in range(len(test_data)):
        y = predict(model, x)
        error_sum += abs(y-test_data[i])
        percent_error_sum += 100*abs(y - test_data[i])/test_data[i]
        if verbose:
            print(
                y, "\t",
                test_data[i], "\t",
                abs(y - test_data[i]), "\t",
                100*abs(y - test_data[i])/test_data[i]
            )
        x = np.array(list(x[1:]) + [sequences[0]['test_data'][i]])
    return {
        "MAE": error_sum/len(test_data),
        "MAPE": percent_error_sum/len(test_data),
    }

errors = []
for i in range(len(sites)):
    errors.append(external_error(vanillas[i], sequences[i]['X'][-1], sequences[i]['test_data']))
    print(f"Site {i+1} :\nerror: {errors[-1]['MAE']}\tmaean value: {np.mean(sequences[i]['test_data'])}\tpercentage error: {errors[-1]['MAPE']}")



Site 1 :
error: 348.7167462158203	maean value: 1704.37	percentage error: 17.258663301274527
Site 2 :
error: 1003.2760565185547	maean value: 486.83	percentage error: 225.10939539299525
Site 3 :
error: 775.8856323242187	maean value: 959.17	percentage error: 129.84998885155696
Site 4 :
error: 1357.3421066474914	maean value: 17.78	percentage error: 9081.935176625742
Site 5 :
error: 1254.9912432861329	maean value: 419.5	percentage error: 334.59158747146694
Site 6 :
error: 2440.5207287597655	maean value: 3320.87	percentage error: 43.31574024930917
Site 7 :
error: 869.3656744384766	maean value: 834.14	percentage error: 120.80357811970751
Site 8 :
error: 797.3396502685547	maean value: 361.14	percentage error: 231.00537932236
Site 9 :
error: 1037.9697387695312	maean value: 438.1	percentage error: 254.1306930436381
Site 10 :
error: 1180.2733209228516	maean value: 463.13	percentage error: 268.96071872679005
Site 11 :
error: 374.916416015625	maean value: 1237.95	percentage error: 29.95768836002264

In [ ]:
A = [15, 1, 11, 6, 53, 66, 73, 74, 96, 86,]
I = np.array(A)-1
data = pd.read_csv("/content/drive/MyDrive/mydata/project/en_wiki.csv")
cols = list(data.columns)
sites = cols[1:]
small_sites = np.array(sites)[I]

cols=["Date"]
for site in small_sites:
    cols.append(site)

new_data = data[cols]
new_data.to_csv("/content/drive/MyDrive/mydata/project/en_wiki_small.csv", index=False)
new_data.head()